In [ ]:
# Run if the code block to train the model is not working and showing errors.
# !pip install --upgrade unsloth transformers torch accelerate bitsandbytes trl
!pip install vllm -q

In [ ]:
# Setting up. Installing unsloth
%%capture
!pip install unsloth -q
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git -q

In [ ]:
# Access your hugging face API
from google.colab import userdata
hf_token = userdata.get() # Use hugging face api key (Or colab secrets)

In [ ]:
# Access your weights and biases API
import wandb
from google.colab import userdata
wb_token = userdata.get()

In [ ]:
# Creates a new project to track experiemnts and fine-tuning process
run = wandb.init(
    project='',
    job_type="training",
    anonymous="allow"
)

In [ ]:
# Loading model and tokenizer
from unsloth import FastLanguageModel

max_seq_length = 1024
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

In [ ]:
# Load Data
from datasets import load_dataset

dataset = load_dataset("json", data_files="QA_finetuned.jsonl", split="train")

train_dataset = dataset.select(range(430))
val_dataset = dataset.select(range(430, 538))

# Example
print(train_dataset)
print(val_dataset)

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def format_example(example):
  """Concatenates each example in the dataset into a single string. Adds a token that tracks the end of a sentence"""
  return {
      "text": f"{example['instruction']}\n\n{example['response']}{EOS_TOKEN}"
  }

train_dataset = train_dataset.map(format_example)
val_dataset = val_dataset.map(format_example)

print(train_dataset)
print(val_dataset)

In [ ]:
# Setting up the model. Adding low rank adapters to the model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

In [ ]:
# Setting up the training arguments and the trainer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        learning_rate=2e-4,
        fp16=False,
        bf16=True,
        logging_steps=10,
        save_strategy="besr",
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="./results",
        load_best_model_at_end=True
    ),
)

In [ ]:
# Model training; run line to start training
trainer_stats = trainer.train()

In [ ]:
from peft import PeftModel

# Step 1: Define your local folder
new_model_local = ""

# Step 2: Merge the LoRA adapters into base model
model = model.merge_and_unload()  # This must come *before* save_pretrained

# Step 3: Save to disk
model.save_pretrained(new_model_local, safe_serialization=True)
tokenizer.save_pretrained(new_model_local)

In [ ]:
# Log into huggingface hub
from huggingface_hub import login
login()

In [ ]:
# Push to huggingface hub
model.push_to_hub("")
tokenizer.push_to_hub("")